In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from xgboost import XGBClassifier
from sklearn.metrics import f1_score, precision_score


In [2]:
# Cargar el conjunto de datos
df = pd.read_csv("datas.csv")

# Separar características (X) y etiquetas (y)
X = df.drop(columns=["adopted"])
y = df["adopted"]

# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:

# Definir el espacio de búsqueda de hiperparámetros
param_space = {
    'learning_rate': np.linspace(0.01, 1.0, 20),
    'max_depth': range(3, 16),
    'n_estimators': range(50, 1001),
    'gamma': np.linspace(0.0, 1.0, 20),
    'subsample': np.linspace(0.5, 1.0, 20),
    'colsample_bytree': np.linspace(0.5, 1.0, 20)
}

# Inicializar el clasificador XGBoost
xgb_clf = XGBClassifier(objective='binary:logistic', random_state=42)

# Inicializar la búsqueda aleatoria con validación cruzada
random_search = RandomizedSearchCV(
    estimator=xgb_clf,
    param_distributions=param_space,
    scoring='f1',
    cv=5,
    n_iter=50,  # número de iteraciones de la búsqueda aleatoria
    random_state=42
)

# Realizar la búsqueda de hiperparámetros
best_f1 = 0
best_precision = 0
print("Búsqueda de hiperparámetros en curso:")
for i in range(1, 51):
    random_search.fit(X_train, y_train)
    
    # Obtener el mejor modelo y sus hiperparámetros
    best_model = random_search.best_estimator_
    best_params = random_search.best_params_

    # Evaluar el mejor modelo en el conjunto de prueba
    y_pred = best_model.predict(X_test)
    f1 = f1_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    
    # Imprimir los resultados
    print(f"Iteración {i}: F1 = {f1}, Precisión = {precision}")
    
    # Guardar el mejor modelo
    if f1 > best_f1:
        best_f1 = f1
        best_precision = precision
        model_name = f"best_xgboost_model_f1_{f1:.4f}_precision_{precision:.4f}.model"
        best_model.save_model(model_name)
        print(f"Mejor modelo guardado como {model_name}")

print("Búsqueda de hiperparámetros finalizada.")